In [1]:
import pandas as pd

In [5]:
Data = pd.read_csv("titanic.csv")
Data.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [6]:
Data.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
Data.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [7]:
inputs = Data.drop('Survived',axis='columns')
target = Data.Survived

#### inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [8]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [9]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


##### I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female

In [10]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [11]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [12]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [13]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [16]:
from sklearn.naive_bayes import GaussianNB
Model = GaussianNB()

In [18]:
Model.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [19]:
Model.score(X_test,y_test)

0.7873134328358209

In [20]:
X_test[0:10]

,Pclass,Age,Fare,female
875,3,15.000000,7.2250,1
469,3,0.750000,19.2583,1
729,3,25.000000,7.9250,1
13,3,39.000000,31.2750,0
229,3,29.699118,25.4667,1
690,1,31.000000,57.0000,0
62,1,45.000000,83.4750,0
369,1,24.000000,69.3000,1
613,3,29.699118,7.7500,0
634,3,9.000000,27.9000,1


In [21]:
y_test[0:10]

875    1
469    1
729    0
13     0
229    0
690    1
62     0
369    1
613    0
634    0
Name: Survived, dtype: int64

In [23]:
Model.predict(X_test[0:10])

array([1, 1, 1, 0, 1, 0, 1, 1, 0, 1], dtype=int64)

In [24]:
Model.predict_proba(X_test[:10])

array([[0.43153208, 0.56846792],
       [0.35168113, 0.64831887],
       [0.47731848, 0.52268152],
       [0.96533081, 0.03466919],
       [0.48364759, 0.51635241],
       [0.56279149, 0.43720851],
       [0.28423295, 0.71576705],
       [0.02317352, 0.97682648],
       [0.96575645, 0.03424355],
       [0.38372511, 0.61627489]])

#### Calculate the score using cross validation

In [25]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.768     , 0.792     , 0.816     , 0.68548387, 0.74193548])